In [1]:
import openpyxl as pl
import numpy as np
from openpyxl import Workbook

In [2]:
# Все используемые функции

def correction(data): # Преображает данные, чтобы по названию бренда можно было сразу достать среднюю цену за 100г
    max_price = -1
    min_price = -1
    for brand in data.keys():
        data[brand] = round(sum(data[brand])/len(data[brand]), 1)
        if min_price == -1 or min_price > data[brand]:
            min_price = data[brand]
        if max_price < data[brand]:
            max_price = data[brand]
    return data, min_price, max_price

def segmentation(data, min_price, max_price): # Выдает массив с брендами, прошедних сегментацию
    for brand in data.keys():
        value = data[brand]
        data[brand] = round((value - min_price) * 100 / (max_price - min_price))
    
    # В словаре data по бренду можно вытащить значение цены за 100г (находится в диапозоне от 0 до 100)
    brands = np.array(list(data.keys()))
    low = select_mas(np.array([data[brand]<25 for brand in brands]), brands) # Применяет функцию select_mas(), в которой находятся булевый массив и массив с брендами
    low.append('Low Priced')
    middle = select_mas(np.array([26<data[brand]<50 for brand in brands]), brands) # Делает то же, что и предыдущий, но для другой сегментации
    middle.append('Middle Priced')
    upper = select_mas(np.array([51<data[brand]<76 for brand in brands]), brands) # Делает то же, что и предыдущий, но для другой сегментации
    upper.append('Upper Middle Priced')
    high = select_mas(np.array([data[brand]>76 for brand in brands]), brands) # Делает то же, что и предыдущий, но для другой сегментации
    high.append('High Priced')
    
    # Создает файл xlsx
    wb = Workbook()
    ws = wb.active
    ws.append(['Бренд', 'Ценовой сегмент'])
    add(low, ws)
    add(middle, ws)
    add(upper, ws)
    add(high, ws)
    wb.save('report.xlsx')
    
    print('Был создан файл report.xlsx, содержащий в себе информацию о брендах и их ценовых сегментах')

def select_mas(mas, brands): # Берет булевый массив и бренды, выдает массив с брендами, которые прошли условие (если в булевом массиве True - условие проходит)
    array_of_brands = [x for x in np.select([mas], [brands]) if x != '0'] # Применяет функцию select(), убирая при этом получившиеся нули
    return array_of_brands

def add(mas, ws): # Добавляет в Workbook строчку с брендами и ценовым сегментом
    for i in range(len(mas)-1):
        ws.append([mas[i], mas[-1]])

In [3]:
# Все необходимые удобно-записанные данные

data = {} # Шаблон: | data = {
          #         |         'Название бренда': [Цены продуктов за 100г одного бренда], 
          #         |         ...
          #         |        }

In [4]:
# Импорт, обработка и запись из файла необходимых нам данных

file = pl.load_workbook('пельмени.xlsx')
# Шаблон xlsx файла: | Название магазина Адрес                      |
#                    | Наименование      Бренд    Цена     Вес      | - (В этой строчке могут быть и другие столбцы.
#                    | (Данные)          (Данные) (Данные) (Данные) |    Которые сейчас написаны - необходимы быть)
#                    | ...                                          |
name = file.get_sheet_names()
for name_one in name:
    sheet = file.get_sheet_by_name(name_one)
    brand_id, price_id, weight_id = -1, -1, -1 # Нужны для выявления столбцов, интересующих нас данных
    for i in range(1, sheet.max_column+1):
        cell = sheet.cell(row = 2, column = i).value
        if cell == 'Бренд':
            brand_id = i
        elif cell == 'Цена':
            price_id = i
        elif cell == 'Вес':
            weight_id = i
    if brand_id != -1 and price_id != -1 and weight_id != -1:
        for i in range(3, sheet.max_row+1):
            brand = sheet.cell(row = i, column = brand_id).value
            price = sheet.cell(row = i, column = price_id).value
            weight = sheet.cell(row = i, column = weight_id).value
            try:
                if brand in data.keys():
                    data[brand].append(round(price/weight*100, 2))
                else:
                    data[brand] = [round(price/weight*100, 2)]
            except:
                print(f'В {name_one} была обнаружена ошибка записи данных в строчке {i}. (Эта строчка была проигнорирована)')
    else:
        print(f'Лист {name_one} не соответствует шаблону заполнения')
data, min_price, max_price = correction(data)
# min_price - Минимальная средняя цена бренда за 100г данного датасета, max_price - Максимальная средняя цена бренда за 100г данного датасета

C:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  
C:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  # Remove the CWD from sys.path while we load stuff.


In [5]:
# Методология сегментации

# У нас есть минимальное и максимальное число в данных. Переведем каждое число бренда в проценты (от 0 до 100).
# 0% - минимальное число, 100% - максимальное число. Формула для перевода: (x - min)*100/(max-min) - это число нахдится в диапозоне от 0 до 100.
# от 0% до 25% - категория Low priced
# от 26% до 50% - категория Middle priced
# от 51% до 75% - категория Upper middle priced
# от 76% до 100% - категория High priced

segmentation(data, min_price, max_price) # Произвела сегментацию и создала эксель файл, разделив бренды на ценовые сегменты

# В реальном кейсе можно было соединить функции correction() и segmentation(), но для наглядности мы их разделили

Был создан файл report.xlsx, содержащий в себе информацию о брендах и их ценовых сегментах
